# Pokemon Database

In [ ]:
import requests
import lxml.html as lh
import pandas as pd

In [ ]:
url = 'https://pokemondb.net/pokedex/all'

In [ ]:
# Query the website and return the html
page = requests.get(url)

# Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [ ]:
# Check the lengh of the first 12 rows to ensure all rows have the same width
# If not, we probably got something more than just the table
[len(T) for T in tr_elements[:12]]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

## 1. Parse Table Header

In [ ]:
tr_elements = doc.xpath('//tr')

# Create empty list
col=[]
i = 0

# For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i, name, '\n')
    col.append((name,[]))

1 # 

2 Name 

3 Type 

4 Total 

5 HP 

6 Attack 

7 Defense 

8 Sp. Atk 

9 Sp. Def 

10 Speed 



Each header is appended to a tuple along with an empty list.

## 2. Creating Pandas DataFrame

In [ ]:
# Starts from 1 as the first row is header so we skip 0
for j in range(1, len(tr_elements)):
    T = tr_elements[j] # the jth row

    # If row is not of size 10, the //tr data is not from our table
    if len(T) != 10:
        break
    i = 0 # the index of our column

    for t in T.iterchildren():
        data = t.text_content()
        if i > 0: # check if the row is empty
            try:
                data = int(data) # convert any numerical value to integers
            except:
                pass
        # Append the data to the empty list of ith column
        col[i][1].append(data)
        # Increment i for the next column
        i += 1

In [ ]:
# Check the length of each column again
[len(c) for (title,c) in col]

[1075, 1075, 1075, 1075, 1075, 1075, 1075, 1075, 1075, 1075]

In [ ]:
# Create the DataFrame
Dict = {title:column for (title, column) in col}
df = pd.DataFrame(Dict)

In [ ]:
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion Female,Water Ghost,530,120,92,65,100,75,78
1071,903,Sneasler,Poison Fighting,510,80,130,60,40,80,120
1072,904,Overqwil,Dark Poison,510,85,115,95,65,65,85
1073,905,Enamorus Incarnate Forme,Fairy Flying,580,74,115,70,135,80,106
